# Computational Mathematics
## An Introduction to Numerical Analysis and Scientific Computing with Python
### By Dimitrios Mitsotakis

# Chapter 5: Appendix

## Complex Step Newton Method

In [1]:
import numpy as np
import numpy.linalg as npl

Here we implement a variant of complex step Newton method. The first part implements the algorithm for scalar equations, as it is described in the book "Computational Mathematics" by D. Mitsotakis. The second part has a variant of this method for systems of nonlinear equations equations.

As unput you need to enter a function `f`, an initial guess `x0`, the number of allowed iterations `maxit` and a tolerance `tol` for detecting convergence. You can also optionally set the small parameter `h`. This can be very small number and by default is `1.e-10`.

The function `cnewton` returns a tuple. The first entry if this tuple is the solution. The number of required iterations and an estimateted error follows.

In [2]:
def cnewton(f,x0,maxit,tol,h=1.e-10):

    # This is an implementaion of the complex step Newton method
    # as it is described in the book "Computational Mathematics"
    # by D. Mitsotakis
    
    error = 1.0
    iters = 0
    
    if type(x0)==float or type(x0)==int:
        x = float(x0)
        while (iters<=maxit and error >= tol):
            deriv=f(x+1j*h)
            x = x - h*f(x)/deriv.imag
            error = np.abs(x-x0)
            x0 = x
            iters += 1
    else:

        x0 = np.array(list(x0))
        n = len(x0)
        x = x0.copy() 
        J = np.zeros((n,n))
        F = f(x)
        if (len(F)!=n):
            raise (InterruptExecution('f must be agree in dimension with x0'))
        while (iters<=maxit and error >= tol):
            for j in range(n):
                xx = x.copy()+1j*0.0
                xx[j]=x[j]+1j*h
                F = f(xx).imag
                for i in range(n):
                    J[i,j]=F[i]
            b1 = f(x)
            b = npl.solve(J,b1)
            x = x - h*b
            xx2 = x-x0
            error = npl.norm(xx2,np.inf)
            x0 = x.copy()
            iters+=1

    return x,iters,error

We test our code for the system:

$$
\begin{aligned}
&x_1+0.25x_2^2=1.25\\
&0.25x_1^2+x_2=1.25
\end{aligned}
$$

which has exact solution $x^\ast=(1,1)$.

In [3]:
def f(x):
    y = np.zeros_like(x)
    y[0]=x[0]+0.25*x[1]**2-1.25
    y[1]=0.25*x[0]**2+x[1]-1.25
    return y

In [4]:
x0 = [0.8,0.8]

xx=cnewton(f,x0,maxit=100,tol=1.e-10,h=1.e-9)

print('Required iterations = ', xx[1])
print('Solution = ', xx[0])
print('Error = ', f(xx[0]))

Required iterations =  4
Solution =  [1. 1.]
Error =  [-2.22044605e-16 -2.22044605e-16]


## Application in Astrophysics

If $\psi$ is the mean anomaly of the orbit of a plant, then $\theta$, the eccentric anomaly, can be computed by solving the fixed point equation
$$\theta=\psi+e\sin \theta$$
where $e$ is the eccentricity of the elliptical orbit.

This equation can be solved seamlesly using the funtion `fixed_point` of `scipy.optimize`.

In [5]:
import numpy as np
import scipy.optimize as spo
def g(theta):
    e = 1.e-6
    psi = np.pi/6.0
    return psi+e*np.sin(theta)
theta0 = np.pi/6.0
theta = spo.fixed_point(g, theta0)
print('eccentric anomaly=', theta)

eccentric anomaly= 0.5235992755987319


Knowing the eccentric anomaly can lead to the estimation of the heliocentric distance $r=a(1-e\cos\theta)$ where $a$ is the semi-major axis.

We repeat the calculation of the previous example using the complex step Newton method as follows

In [6]:
def f(theta):
    return theta-g(theta)

In [7]:
x0 = 1.0
maxit = 100
tol = 1.e-9
h = 1.e-9

theta = cnewton(f,x0,maxit,tol,h=1.e-10)

print('Required iterations = ', theta[1])
print('eccentric anomaly =', theta[0])
print('Error = ', f(theta[0]))

Required iterations =  3
eccentric anomaly = 0.5235992755987319
Error =  0.0
